In [50]:
# 라이브러리 로드
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [51]:
# 데이터 로드

수익성 = pd.read_csv('./data/코스닥/코스닥 수익성.csv',encoding='cp949')
안정성 = pd.read_csv('./data/코스닥/코스닥 안정성.csv',encoding='cp949')
활동성 = pd.read_csv('./data/코스닥/코스닥 활동성.csv',encoding='cp949')
성장성 = pd.read_csv('./data/코스닥/코스닥 성장성 계산용.csv',encoding='cp949')
현금흐름 = pd.read_csv('./data/코스닥/코스닥 현금흐름 + 누적수익성비율.csv',encoding='cp949')
연구개발 = pd.read_csv('./data/코스닥/코스닥 rs.csv',encoding='cp949')
EBIT = pd.read_csv('./data/코스닥/에비~ 코스닥.csv',encoding='cp949')
per_pbr_총부채 = pd.read_csv('./data/코스닥/per pbr 총부채 코스닥.csv',encoding='cp949')
직원수_업력 = pd.read_csv('./data/코스닥/코스닥 직원수 급여 업력 외국인 대주주.csv',encoding='cp949')
시가총액 = pd.read_csv('./data/코스닥/코스닥시가총액.csv',index_col=0)  #### 시총도 붙여야됨
단기차입금 = pd.read_csv('./data/코스닥/코스닥 단기차입금 유동성 장기부채.csv',encoding='cp949')
거시변수 = pd.read_excel('./data/거시경제 변수/거시변수.xlsx', engine="openpyxl")
상장일 = pd.read_csv('./data/코스닥/코스닥 상장일.csv',encoding='cp949')
대분류 = pd.read_csv('./data/코스닥/코스닥 대분류.csv',encoding='cp949')
유형자산 = pd.read_csv("./data/코스닥/코스닥 유형자산증가율.csv", encoding= "cp949")

In [52]:
#  사용 안하는 코드 컬럼 삭제 
대분류.drop(['통계청 한국표준산업분류 코드 10차(대분류)'],axis=1,inplace=True)

In [53]:
# 데이터 결합

merge_data = [안정성, 수익성, 활동성, 성장성, 현금흐름, 연구개발, EBIT, per_pbr_총부채, 직원수_업력, 단기차입금, 상장일, 대분류, 유형자산]

df = 안정성

for data in merge_data[1:]:
    df = df.merge(data, on=['회사명', '거래소코드', '회계년도'], how='left')


In [54]:
pd.reset_option("display.max.rows", None)
print('전체 행:', df.shape[0],'개')
print('전체 기업 수',len(df['회사명'].unique()))
print(df.isna().sum())

전체 행: 15617 개
전체 기업 수 1744
회사명                            0
거래소코드                          0
회계년도                           0
자산(*)(요약)(IFRS)             1844
자본(*)(요약)(IFRS)             1844
                            ... 
유동성장기부채(요약)(IFRS연결)(백만원)    5530
상장일                            0
통계청 한국표준산업분류 10차(대분류)          0
유형자산의 증가(*)(IFRS)(천원)       1844
유형자산의 증가(*)(IFRS연결)(천원)     5530
Length: 97, dtype: int64


In [55]:
# 거래소 코드 6자리로 맞춰주는 함수
def pad_code(x):
    df[x] = df[x].astype(str).str.rjust(6, "0")
    # return df

# 외국계 기업 삭제 함수
def remove_foreign_companies(df):
    foreign_cp_ind = df[df["거래소코드"].str[0] == "9"].index
    print("외국계기업 데이터 수: ", len(foreign_cp_ind))
    print("외국계기업 수: ",len(np.unique(df[df["거래소코드"].astype("str").str[0] == "9"]["회사명"])))
    df.drop(index=foreign_cp_ind, axis=0, inplace=True)
    df.reset_index(drop=True, inplace=True)
    print('남은 데이터 수',df.shape[0]) 
    print('남은 기업 수',len(df['회사명'].unique()))
    # return df

# 회계년도 컬럼에서 년도만 추출 후 int화 함수 
def change_year(df):
    df["회계년도"] = df["회계년도"].astype("str").str[:4].astype("int")
    # return df

# 중복되는 데이터 확인 후 삭제하는 함수
def find_duplicate_rows(df):
    duplicate_rows = df[df.duplicated() == True].index
    print("중복되는 데이터 수 : ",len(duplicate_rows))
    df.drop(index = duplicate_rows ,axis=0, inplace=True)
    df.reset_index(drop=True, inplace=True)
    # return df

# 중복되는 거래소 코드 확인 후 하나의 거래소 코드만 남기는 함수 
def del_duplicated_code(df):
    if len(df["회사명"].unique()) != len(df["거래소코드"].unique()):
        print("회사수:", len(df["회사명"].unique()))
        print("거래소코드수:", len(df["거래소코드"].unique()))
        cp_ls = df["회사명"].unique()
        duplicated_ls = [c for c in cp_ls if df[df["회사명"] == c]["거래소코드"].nunique() > 1]
        print("거래소코드가 중복되는 기업 수:", len(duplicated_ls))
        # for i in duplicated_ls:
        #     print(i)
        # 중복 거래소코드 제거
        for company_name in duplicated_ls:
            # 각 기업의 데이터프레임 추출
            df_company = df[df["회사명"] == company_name]

            # 마지막 거래소 코드만 남기기
            last_code = df_company["거래소코드"].iloc[-1]
            df_company = df_company[df_company["거래소코드"] == last_code]

            # 중복 제거한 데이터프레임을 원래 데이터프레임에 업데이트
            df = pd.concat([df[df["회사명"] != company_name], df_company], ignore_index=True)
        
            
        print("중복 거래소 코드 제거 후 회사수:", len(df["회사명"].unique()))
        print("중복 거래소 코드 제거 후 거래소 코드 수:", len(df["거래소코드"].unique()))
        print('남은 데이터 수',df.shape[0]) 
        print('남은 기업 수',len(df['회사명'].unique()))

    elif len(df["회사명"].unique()) != len(df["거래소코드"].unique()):
        print("회사수:", len(df["회사명"].unique()))
        print("거래소코드수:", len(df["거래소코드"].unique()))
    
    return df

#  연결 재무제표를 개별제무제표로 대체하는 함수
def replace_yeongyeol_with_gaebyeol(df):
    connected_cols = [col for col in df.columns if "연결" in col]

    for connected_col in connected_cols:
        original_col = connected_col.replace("연결", "")
        if original_col in df.columns:
            df.loc[df[connected_col].isnull(), connected_col] = df[original_col]

    df["차입금의존도(IFRS연결)"].fillna(df["차입금의존도(IFRS)(백만원)"], inplace=True)
    df["유보율(IFRS연결)(백만원)"].fillna(df["유보율(IFRS)"], inplace=True)
    df["현금비율(IFRS연결)(백만원)"].fillna(df["현금비율(IFRS)"], inplace=True)
        
    # return df



In [56]:
# 거래소 코드 6자리로 맞춰 주기
pad_code("거래소코드")

# 회계년도에서 년도만 추출 후 int화
change_year(df)

# 중복되는 데이터 확인후 삭제
find_duplicate_rows(df)

# 외국계 기업 삭제
remove_foreign_companies(df)

중복되는 데이터 수 :  0
외국계기업 데이터 수:  152
외국계기업 수:  23
남은 데이터 수 15465
남은 기업 수 1721


In [57]:
# 거래소 코드가 중복되는 데이터 확인 후 하나의 거래소 코드만 남기기
df = del_duplicated_code(df)

회사수: 1721
거래소코드수: 1743
거래소코드가 중복되는 기업 수: 22
중복 거래소 코드 제거 후 회사수: 1721
중복 거래소 코드 제거 후 거래소 코드 수: 1721
남은 데이터 수 15329
남은 기업 수 1721


In [58]:
# # 연결 재무재표의 결측치 개별 재무재표의 채우기
replace_yeongyeol_with_gaebyeol(df)

In [59]:
df.isna().sum()

회사명                            0
거래소코드                          0
회계년도                           0
자산(*)(요약)(IFRS)             1667
자본(*)(요약)(IFRS)             1667
                            ... 
유동성장기부채(요약)(IFRS연결)(백만원)    1666
상장일                            0
통계청 한국표준산업분류 10차(대분류)          0
유형자산의 증가(*)(IFRS)(천원)       1667
유형자산의 증가(*)(IFRS연결)(천원)     1666
Length: 97, dtype: int64

In [60]:
pd.reset_option("display.max.rows", None)
print('전체 행:', df.shape[0],'개')
print('전체 기업 수',len(df['회사명'].unique()))
print(df.isna().sum())

전체 행: 15329 개
전체 기업 수 1721
회사명                            0
거래소코드                          0
회계년도                           0
자산(*)(요약)(IFRS)             1667
자본(*)(요약)(IFRS)             1667
                            ... 
유동성장기부채(요약)(IFRS연결)(백만원)    1666
상장일                            0
통계청 한국표준산업분류 10차(대분류)          0
유형자산의 증가(*)(IFRS)(천원)       1667
유형자산의 증가(*)(IFRS연결)(천원)     1666
Length: 97, dtype: int64


In [61]:
#  삭제 할 개별재무제표 컬럼 정리  
pd.reset_option("display.max.rows", None)

drop_col = ['유보율(IFRS)' ,'[제조]* (정상)영업손익(보고서기재)(요약)(IFRS)(백만원)','[제조]      이익잉여금(결손금)(요약)(IFRS)(백만원)',
'[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS)(백만원)','자산(*)(요약)(IFRS)(백만원)', 
'부채(*)(요약)(IFRS)(백만원)','[제조]자본(*)(요약)(IFRS)(백만원)','당기순이익(손실)(요약)(IFRS)(백만원)', 
'법인세비용차감전순이익(손실)(요약)(IFRS)(백만원)','금융원가(요약)(IFRS)(백만원)','금융수익(요약)(IFRS)(백만원)',
'매출액(수익)(요약)(IFRS)(백만원)','연구개발비(IFRS)(천원)',
'[제조]      유형자산(요약)(IFRS)(백만원)','자본금회전률(IFRS)',
'총자본회전률(IFRS)','재고자산회전률(IFRS)','유형자산회전율(IFRS)','자기자본회전률(IFRS)','타인자본회전률(IFRS)',
'경영자본순이익률(IFRS)','금융비용부담률(IFRS)','총자본순이익률(IFRS)','매출액총이익률(IFRS)',
'자기자본순이익률(IFRS)','총자본정상영업이익률(IFRS)','자산(*)(요약)(IFRS)','자본(*)(요약)(IFRS)',
 '현금비율(IFRS)','* (정상)영업손익(보고서기재)(요약)(IFRS)(백만원)',
 '차입금의존도(IFRS)(백만원)','감가상각비(IFRS)(천원)','부채비율(IFRS)',
'당좌비율(IFRS)','*이자비용(요약)(IFRS)(백만원)','순운전자본비율(IFRS)',
'[제조]자산(*)(요약)(IFRS)(백만원)_x','[제조]자산(*)(요약)(IFRS연결)(백만원)_x',
'[제조]자산(*)(요약)(IFRS)(백만원)_y','[제조]자산(*)(요약)(IFRS연결)(백만원)_y',
'[제조]매출액(수익)(요약)(IFRS)(백만원)_y', '[제조]매출액(수익)(요약)(IFRS연결)(백만원)_y',
'[제조]매출액(수익)(요약)(IFRS)(백만원)_x', '[제조]매출액(수익)(요약)(IFRS연결)(백만원)_x',
'유동성장기부채(요약)(IFRS)(백만원)','단기차입금(요약)(IFRS)(백만원)','자산(*)(요약)(IFRS연결)(백만원)','[제조]자본(*)(요약)(IFRS연결)(백만원)',"유형자산의 증가(*)(IFRS)(천원)"
]

# 개별재무제표 컬럼 삭제
df.drop(drop_col,axis=1,inplace=True)

In [62]:
## 성장성 지표인 증가율 컬럼 만들기
성장성_col = ['[제조]      유형자산(요약)(IFRS연결)(백만원)','매출액(수익)(요약)(IFRS연결)(백만원)',
           '자본(*)(요약)(IFRS연결)','자산(*)(요약)(IFRS연결)',
           '[제조]* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)']

## 증가율 컬럼 전기에 음수일때 당기에 음수지만 증가할때 증가율 계산
## ex (2011년 -20, 2012년 -10) -> -10-(-20) / np.abs(-20)= 
증가율_df = (df.groupby(['회사명', '거래소코드'])[성장성_col].shift(0) - df.groupby(['회사명', '거래소코드'])[성장성_col].shift(1)) / np.abs(df.groupby(['회사명', '거래소코드'])[성장성_col].shift(1))*100
증가율_df.rename(columns={'[제조]      유형자산(요약)(IFRS연결)(백만원)':'유형자산증가율', '매출액(수익)(요약)(IFRS연결)(백만원)':'매출액증가율',
       '자본(*)(요약)(IFRS연결)':'자기자본증가율', '자산(*)(요약)(IFRS연결)':'총자본증가율',
       '[제조]* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)':'정상영업이익증가율'},inplace=True)

df.drop(['[제조]      유형자산(요약)(IFRS연결)(백만원)','[제조]* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)'],axis=1,inplace=True)
df = pd.concat([df,증가율_df],axis=1)

In [63]:
## 시가총액 merge
change_year(시가총액)
df = df.merge(시가총액,on=['거래소코드','회계년도'],how='left')
df['시가총액(백만원)'] = df['시가총액'] / 1000000
df.drop(['시가총액'],axis=1,inplace=True)

## 거시변수 merge
거시변수.rename(columns={'회계연도':'회계년도'},inplace=True)
change_year(거시변수)
df = df.merge(거시변수,on=['회계년도'],how='left')

In [64]:
### 파생변수 만들기 


## 기업별 업력 설립일 이후 지속된 기간
df['업력'] = df['회계년도'] -  pd.to_datetime(df['설립일'],format='%Y/%m/%d').dt.strftime('%Y').astype(int)
df.drop('설립일',axis=1,inplace=True)

## 누적수익성비율  - 자산0이면 inf - 공시안함 
df['누적수익성비율'] = (df['[제조]      이익잉여금(결손금)(요약)(IFRS연결)(백만원)']/df['자산(*)(요약)(IFRS연결)']) * 100

# 이자부담율
df['이자부담률'] = (df['EBITDA(IFRS)(백만원)'] / df['*이자비용(요약)(IFRS연결)(백만원)']) * 100 

# 부채상환계수 EBITDA / (이자비용 + 단기차입금 + 유동성장기부채)
df['부채상환계수'] =( df['EBITDA(IFRS)(백만원)'] / (df['*이자비용(요약)(IFRS연결)(백만원)']+df['단기차입금(요약)(IFRS연결)(백만원)'] + df['유동성장기부채(요약)(IFRS연결)(백만원)'])) * 100

# 현금흐름_자산 = 영업현금흐름 / 당기총자산
df['현금흐름 대 자산'] = (df.groupby(['회사명','거래소코드'])['[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS연결)(백만원)'].shift(0) / df.groupby(['회사명','거래소코드'])['자산(*)(요약)(IFRS연결)'].shift(0)) * 100

# 현금흐름_매출액
df['현금흐름 대 매출액'] = (df['[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS연결)(백만원)'] / df['매출액(수익)(요약)(IFRS연결)(백만원)']) * 100

# 토빈Q
df['토빈Q'] = ((df['부채(*)(요약)(IFRS연결)(백만원)'] + df['시가총액(백만원)']) / df['자산(*)(요약)(IFRS연결)']) * 100 

## TMD
df['TMD'] =( df['부채(*)(요약)(IFRS연결)(백만원)'] / df['시가총액(백만원)']) * 100

# 알트만 변수
df['시총/총자산'] = (df['시가총액(백만원)'] / df['자산(*)(요약)(IFRS연결)']) *100

# RD -단위도 수정
df['연구개발비(IFRS연결)(백만원)'] = df['연구개발비(IFRS연결)(천원)']/1000
df.drop(['연구개발비(IFRS연결)(천원)'],axis=1,inplace=True)
df['RD'] = (df['연구개발비(IFRS연결)(백만원)'] / df['매출액(수익)(요약)(IFRS연결)(백만원)']) * 100

# 직원수변동률 , 직원인건비총액(백만원단위) (고민필요)
df['직원수변동률%'] = df.groupby(['회사명','거래소코드'])['합계_기말인원(명)'].pct_change()*100

# 직원인건비총액(백만원)
df['직원인건비총액(백만원)'] = (df['합계_년간총급여(천원)']/1000) * df['합계_기말인원(명)']
df.drop(['합계_년간총급여(천원)'],axis=1,inplace=True)

# 알트만 변수  EBIT = 법인세비용차감전순이익+금융원가-금융수익
df['EBIT/총자산'] = ((df['법인세비용차감전순이익(손실)(요약)(IFRS연결)(백만원)'] + df['금융원가(요약)(IFRS연결)(백만원)']-df['금융수익(요약)(IFRS연결)(백만원)'])/df['자산(*)(요약)(IFRS연결)']) * 100

# PBR 
df['PBR'] = (df['시가총액(백만원)']  / df['자본(*)(요약)(IFRS연결)']) * 100


# 영업손익 전기영업손익 + 당기영업손익
df["당기전기영업손익"] = (df.groupby(["회사명", "거래소코드"])["* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)"].shift(0) + df.groupby(["회사명", "거래소코드"])["* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)"].shift(1)) / 2
df['당기전기매출액'] = (df.groupby(["회사명", "거래소코드"])["매출액(수익)(요약)(IFRS연결)(백만원)"].shift(0) + df.groupby(["회사명", "거래소코드"])["매출액(수익)(요약)(IFRS연결)(백만원)"].shift(1)) / 2
df["당기전기영업손익/매출액"] = (df['당기전기영업손익'] / df['당기전기매출액']) * 100

# capex컬럼 단위 맞춰주기
df["유형자산의 증가(*)(IFRS연결)(천원)"] = df["유형자산의 증가(*)(IFRS연결)(천원)"] / 1000

# findep =  (capex - 영업현금흐름 ) / capex
df["FINDEP"] = (df["유형자산의 증가(*)(IFRS연결)(천원)"] - df['[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS연결)(백만원)']) *100

# PBR 변화율
df['PBR변화율']  = (df.groupby(['회사명', '거래소코드'])['PBR'].shift(0) - df.groupby(['회사명', '거래소코드'])['PBR'].shift(1)) / np.abs(df.groupby(['회사명', '거래소코드'])['PBR'].shift(1)) *100

# ROA 변화율
df['ROA변화율']  = (df.groupby(['회사명', '거래소코드'])['총자본순이익률(IFRS연결)'].shift(0) - df.groupby(['회사명', '거래소코드'])['총자본순이익률(IFRS연결)'].shift(1)) / np.abs(df.groupby(['회사명', '거래소코드'])['총자본순이익률(IFRS연결)'].shift(1)) *100

# EBITDA마진율
df["EBITDA마진율"] = (df["EBITDA(IFRS)(백만원)"] / df["매출액(수익)(요약)(IFRS연결)(백만원)"]) * 100

# 논문을 바탕으로 새로운 컬럼 생성

df['현금흐름/총부채비율'] = (df['[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS연결)(백만원)'] / df['부채(*)(요약)(IFRS연결)(백만원)']) * 100
df["영업현금흐름-단기차입금"] = (df["[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS연결)(백만원)"] - df["단기차입금(요약)(IFRS연결)(백만원)"]) * 100
df["CND"] =( np.abs((df["[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS연결)(백만원)"] - df["당기순이익(손실)(요약)(IFRS연결)(백만원)"])) / df["매출액(수익)(요약)(IFRS연결)(백만원)"]) *100


In [65]:
# 컬럼이름 정리

rename_list = {'자산(*)(요약)(IFRS연결)':'자산', '자본(*)(요약)(IFRS연결)':'자본',
       '차입금의존도(IFRS연결)':'차입금의존도', '유보율(IFRS연결)(백만원)':'유보율', '현금비율(IFRS연결)(백만원)':'현금비율',
       '부채비율(IFRS연결)':'부채비율', '당좌비율(IFRS연결)':'당좌비율', '순운전자본비율(IFRS연결)':'순운전자본비율',
       '*이자비용(요약)(IFRS연결)(백만원)':'이자비용', '* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)':'영업손익',
       '감가상각비(IFRS연결)(천원)':'감가상각비', 'EBITDA(IFRS)(백만원)':'EBITDA', '총자본정상영업이익률(IFRS연결)':'총자본정상영업이익률',
       '자기자본순이익률(IFRS연결)':'자기자본순이익률', '총자본순이익률(IFRS연결)':'총자본순이익률', '매출액총이익률(IFRS연결)':'매출액총이익률',
       '금융비용부담률(IFRS연결)':'금융비용부담률', '경영자본순이익률(IFRS연결)':'경영자본순이익률', '자본금회전률(IFRS연결)':'자본금회전률',
       '유형자산회전율(IFRS연결)':'유형자산회전율', '재고자산회전률(IFRS연결)':'재고자산회전률', '자기자본회전률(IFRS연결)':'자기자본회전률',
       '타인자본회전률(IFRS연결)':'타인자본회전률', '총자본회전률(IFRS연결)':'총자본회전률', 
       '[제조]      이익잉여금(결손금)(요약)(IFRS연결)(백만원)':'이익잉여금',
       '[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS연결)(백만원)':'영업활동으로 인한 현금흐름',
       '매출액(수익)(요약)(IFRS연결)(백만원)':'매출액', '금융원가(요약)(IFRS연결)(백만원)':'금융원가',
       '금융수익(요약)(IFRS연결)(백만원)':'금융수익', '법인세비용차감전순이익(손실)(요약)(IFRS연결)(백만원)':'법인세비용차감전순이익',
       '당기순이익(손실)(요약)(IFRS연결)(백만원)':'당기순이익', 
       '부채(*)(요약)(IFRS연결)(백만원)':'부채', '외국인_주식분포비율':'외국인지분율', '보통주식비율':'대주주지분율',
       '단기차입금(요약)(IFRS연결)(백만원)':'단기차입금', '유동성장기부채(요약)(IFRS연결)(백만원)':'유동성장기부채',
       '시가총액(백만원)':'시가총액','연구개발비(IFRS연결)(백만원)':'연구개발비','직원인건비총액(백만원)':'직원인건비총액','통계청 한국표준산업분류 10차(대분류)':'산업군',
       "유형자산의 증가(*)(IFRS연결)(천원)" : "CAPEX"
       }

df.rename(columns=rename_list,inplace=True)

In [66]:
## 회계년도 2011년도 제거
df['상장일'] = pd.to_datetime(df['상장일'],format='%Y/%m/%d').dt.strftime('%Y').astype(int)


df = df[df['회계년도']>2011].reset_index(drop=True)

print('전체 행:', df.shape[0],'개')
print('전체 기업 수',len(df['회사명'].unique()))


전체 행: 14077 개
전체 기업 수 1684


-----

## 타겟 라벨링
-----


In [67]:
#  데이터 로드, 필요 없는 컬럼 삭제

부실 = pd.read_csv('./data/y/y값최종.csv',index_col=0)
change_year(부실)
부실["거래소코드"] = 부실["거래소코드"].astype("str").str.rjust(6,"0")
부실.drop(['번호','회사명'],axis=1,inplace=True)

In [68]:
# 데이터 결합
df = df.merge(부실,on=['회계년도','거래소코드'],how='left')

In [69]:
## 상폐기업 라벨링

df["공시제목"].fillna(0,inplace=True)

상폐_ind = df[df["공시제목"]== '상장폐지기업'].index

### set1
df.loc[상폐_ind,"target_1"] = 1

for i in df.index:
    if (df.loc[i,"target_1"] == 1) and (df.loc[i,"회사명"] == df.loc[i-1,"회사명"]) :
        df.loc[i-1, "target_1"] = 1
df['target_1'].fillna(0,inplace=True)
df['target_1'] = df['target_1'].astype(int)

# ### set2
df.loc[상폐_ind,"target_2"] = 1

for i in df.index:
    if (df.loc[i,"target_2"] == 1) and (df.loc[i,"회사명"] == df.loc[i-1,"회사명"]) and (df.loc[i,'공시제목']=='상장폐지기업') :
        df.loc[i-1, "target_2"] = 1 
    if (df.loc[i,"target_2"] == 1) and (df.loc[i,"회사명"] == df.loc[i-2,"회사명"]) and (df.loc[i,'공시제목']=='상장폐지기업'):
        df.loc[i-2, "target_2"] = 1

df['target_2'].fillna(0,inplace=True)
df['target_2'] = df['target_2'].astype(int)

## set_3

df.loc[상폐_ind,"target_3"] = 1

for i in df.index:
    if (df.loc[i,"target_3"] == 1) and (df.loc[i,"회사명"] == df.loc[i-1,"회사명"]) and (df.loc[i,'공시제목']=='상장폐지기업'):
        df.loc[i-1, "target_3"] = 1
df['target_3'].fillna(0,inplace=True)
df['target_3'] = df['target_3'].astype(int)


In [70]:
print(df.target_1.sum())
print(df.target_2.sum())
print(df.target_3.sum())

237
331
237


In [71]:
## 회생기업 라벨링

df["공시제목"].fillna(0,inplace=True)

회생_ind = df[df["공시제목"]== '회생절차개시결정'].index

### set1
df.loc[회생_ind,"target_1"] = 1

for i in df.index:
    if (df.loc[i,"target_1"] == 1) and (df.loc[i,"회사명"] == df.loc[i-1,"회사명"]) & (df.loc[i,'공시제목']=='회생절차개시결정'):
        df.loc[i-1, "target_1"] = 1
df['target_1'].fillna(0,inplace=True)
df['target_1'] = df['target_1'].astype(int)

# ### set2
df.loc[회생_ind,"target_2"] = 1

df['target_2'].fillna(0,inplace=True)
df['target_2'] = df['target_2'].astype(int)

## set_3

df.loc[회생_ind,"target_3"] = 1

df['target_3'].fillna(0,inplace=True)
df['target_3'] = df['target_3'].astype(int)

In [72]:
print(df.target_1.sum())
print(df.target_2.sum())
print(df.target_3.sum())

312
369
275


In [73]:
print("부실 기업 수_1  : ",len(np.unique(df[df["target_1"]==1]["회사명"])))
print("부실 기업 수_2  : ",len(np.unique(df[df["target_2"]==1]["회사명"])))
print("부실 기업 수_3  : ",len(np.unique(df[df["target_3"]==1]["회사명"])))

부실 기업 수_1  :  167
부실 기업 수_2  :  167
부실 기업 수_3  :  167


In [74]:
print("전체 데이터 수 : ", df.shape[0])
print("부실 기업 데이터 수: ",len(df[df["target_3"]==1]))
print("정상 기업 데이터 수 : ",len(df[df["target_3"]==0]))
print("부실 정상 데이터 비율 : ",df["target_3"].value_counts(property))
print("전체 기업 수 : " , len(np.unique(df["회사명"])))
print("부실 기업 수  : ",len(np.unique(df[df["target_1"]==1]["회사명"])))
print("정상 기업 수 : ",len(np.unique(df[df["target_3"]==0]["회사명"])))
print("결측치 수 : ", df.isna().sum().sum())


전체 데이터 수 :  14077
부실 기업 데이터 수:  275
정상 기업 데이터 수 :  13802
부실 정상 데이터 비율 :  target_3
0    0.980465
1    0.019535
Name: proportion, dtype: float64
전체 기업 수 :  1684
부실 기업 수  :  167
정상 기업 수 :  1649
결측치 수 :  121027


In [75]:
def get_index_with_50_or_more_nan_columns(df):
    
    # 결측치 컬럼 개수 계산
    nan_column_counts = df.isnull().sum(axis=1)
    
    # 결측치 컬럼 개수가 50개 이상인 인덱스 번호 추출
    index_with_50_or_more_nan_columns = nan_column_counts[nan_column_counts >= 50].index.to_list()
    print('제거행 수:' , len(index_with_50_or_more_nan_columns))

    df.drop(index=index_with_50_or_more_nan_columns,inplace=True)
    return df


In [76]:
print(len(get_index_with_50_or_more_nan_columns(df)))
print('전체 기업 수',len(df['회사명'].unique()))


제거행 수: 1455
12622
전체 기업 수 1632


In [77]:
print('부실데이터 수:',df[df['target_1']==1].shape[0])
print('부실데이터 수:',df[df['target_2']==1].shape[0])
print('부실데이터 수:',df[df['target_3']==1].shape[0])

print("전체 데이터 수 : ", df.shape[0])
print("부실 기업 데이터 수: ",len(df[df["target_3"]==1]))
print("정상 기업 데이터 수 : ",len(df[df["target_3"]==0]))
print("부실 정상 데이터 비율 : ",df["target_3"].value_counts(property))
print("전체 기업 수 : " , len(np.unique(df["회사명"])))
print("부실 기업 수  : ",len(np.unique(df[df["target_1"]==1]["회사명"])))
print("정상 기업 수 : ",len(np.unique(df[df["target_3"]==0]["회사명"])))
print("결측치 수 : ", df.isna().sum().sum())

부실데이터 수: 279
부실데이터 수: 325
부실데이터 수: 245
전체 데이터 수 :  12622
부실 기업 데이터 수:  245
정상 기업 데이터 수 :  12377
부실 정상 데이터 비율 :  target_3
0    0.980589
1    0.019411
Name: proportion, dtype: float64
전체 기업 수 :  1632
부실 기업 수  :  151
정상 기업 수 :  1596
결측치 수 :  23906


In [78]:
pd.set_option('display.max_rows',None)
print(df.isna().sum())
pd.reset_option('display.max_rows',None)

회사명                  0
거래소코드                0
회계년도                 0
자산                   1
자본                   1
차입금의존도             186
유보율                186
현금비율               186
부채비율               186
당좌비율               186
순운전자본비율            186
이자비용                 1
영업손익                 1
감가상각비                1
EBITDA             188
총자본정상영업이익률         186
자기자본순이익률           186
총자본순이익률            186
매출액총이익률            186
금융비용부담률            186
경영자본순이익률           186
자본금회전률             186
유형자산회전율            186
재고자산회전률            186
자기자본회전률            186
타인자본회전률            186
총자본회전률             186
이익잉여금                1
영업활동으로 인한 현금흐름       1
매출액                  1
금융원가                 1
금융수익                 1
법인세비용차감전순이익          1
당기순이익                1
부채                   1
합계_기말인원(명)        1085
외국인지분율            1351
대주주지분율            1095
단기차입금                1
유동성장기부채              1
상장일                  0
산업군                  0
CAPEX                1
유형자산증가율    

In [79]:
# 상장일이 회계년도 보다 작은 데이터 삭제
df = df[df["회계년도"] > df["상장일"]]
df.reset_index(drop = True, inplace = True)

print("전체 데이터 수 : ", df.shape[0])
print("부실 기업 데이터 수: ",len(df[df["target_3"]==1]))
print("정상 기업 데이터 수 : ",len(df[df["target_3"]==0]))
print("부실 정상 데이터 비율 : ",df["target_3"].value_counts(property))
print("전체 기업 수 : " , len(np.unique(df["회사명"])))
print("부실 기업 수  : ",len(np.unique(df[df["target_1"]==1]["회사명"])))
print("정상 기업 수 : ",len(np.unique(df[df["target_3"]==0]["회사명"])))
print("결측치 수 : ", df.isna().sum().sum())

전체 데이터 수 :  10807
부실 기업 데이터 수:  243
정상 기업 데이터 수 :  10564
부실 정상 데이터 비율 :  target_3
0    0.977515
1    0.022485
Name: proportion, dtype: float64
전체 기업 수 :  1473
부실 기업 수  :  150
정상 기업 수 :  1437
결측치 수 :  1277


In [80]:
# 이용한 컬럼 삭제 

df.drop(['자산','이자비용','매출액','부채','금융원가','시가총액',"EBITDA",'공시제목','상장일'],axis=1,inplace=True)

In [81]:
# 이자부당률이 결측치로 나오는 14년도 영진코퍼레이션 결측치 직접 값 계산해서 결측치 대체
df.loc[df[df["이자부담률"].isna()].index, "이자부담률"] = -5.995505908521291

# 부채상환계수rk 결측치로 나오는 14년도 영진코퍼레이션 결측치 직접 값 계산해서 결측치 대체
df.loc[df[df["부채상환계수"].isna()].index, "부채상환계수"] = -760.5303077054485

# 대주주 지분율 없는 컬럼 다트에서 값 찾아서 대체 

df.loc[903, "대주주지분율"] = 50                    # 드림티엔터테인먼트
df.loc[2904, "대주주지분율"] = 27.83                # 아라온테크
df.loc[3643, "대주주지분율"] = 49.8                 # 에이루트
df.loc[3677, "대주주지분율"] = 18.5                 # 에이제이에스
df.loc[4368, "대주주지분율"] = 23.84                # 와이즈파워
df.loc[5496, "대주주지분율"] = 100                  # 제이테크놀로지
df.loc[8094, "대주주지분율"] = 12.03                # 대한그린파워
df.loc[8265, "대주주지분율"] = 14.94                # 디브이에스코리아
df.loc[8746, "대주주지분율"] = 40.78                # 세미텍


#  유형자산증가율 없는 컬럼 다트에서 값 찾아서 대체 
df["유형자산증가율"].fillna(0, inplace=True)        # 비츠로시스


# 외국인 지분율 결측치 0으로 대체
df["외국인지분율"].fillna(0, inplace=True)


# 직원인건비총액 결측치 0으로 대체
df["직원인건비총액"].fillna(0, inplace=True)

# 합계_기말인원(명) 결측치 0으로 대체
df["합계_기말인원(명)"].fillna(0, inplace=True)



#  유형자산증가율 없는 컬럼 다트에서 값 찾아서 대체 
df.loc[1879, "유형자산증가율"] = 0                    # # 유형자산을 모두 처분하여 감가상각비가 없어 EBITDA값이 결측치인 14년도 영진코퍼레이션 결측치에 영업이익값으로 대체

# #매출액증가율 없는 컬럼 삭제

de_ind_1 = df[df[["매출액증가율"]].isna().any(axis =1)].index
print("매출액증가율 삭제 행 수 : ",len(de_ind_1))
df.drop(index=de_ind_1, axis=0, inplace=True)
 # 메드팩토 셀리드 큐리언트


 # 증가율 컬럼 당해 회계년도에 상장하여 전년도 자료가 없어 결측치로 뜨는 데이터 삭제   
de_ind = df[df[["총자본증가율"]].isna().any(axis =1)].index
print("총자본증가율 삭제 행 수 : ",len(de_ind))
df.drop(index=de_ind, axis=0, inplace=True)

# 직원수변동률 결측치 0으로 대체 
de_ind_2 = df[df[["직원수변동률%"]].isna().any(axis =1)].index
print("직원수변동률 삭제 행 수 : ",len(de_ind_2))
df.drop(index=de_ind_2, axis=0, inplace=True)
#  비덴트 썬테크놀로지스

매출액증가율 삭제 행 수 :  119
총자본증가율 삭제 행 수 :  0
직원수변동률 삭제 행 수 :  3


In [82]:
pd.set_option('display.max_rows',None)
print(df.isna().sum())
pd.reset_option('display.max_rows',None)

회사명                0
거래소코드              0
회계년도               0
자본                 0
차입금의존도             0
유보율                0
현금비율               0
부채비율               0
당좌비율               0
순운전자본비율            0
영업손익               0
감가상각비              0
총자본정상영업이익률         0
자기자본순이익률           0
총자본순이익률            0
매출액총이익률            0
금융비용부담률            0
경영자본순이익률           0
자본금회전률             0
유형자산회전율            0
재고자산회전률            0
자기자본회전률            0
타인자본회전률            0
총자본회전률             0
이익잉여금              0
영업활동으로 인한 현금흐름     0
금융수익               0
법인세비용차감전순이익        0
당기순이익              0
합계_기말인원(명)         0
외국인지분율             0
대주주지분율             0
단기차입금              0
유동성장기부채            0
산업군                0
CAPEX              0
유형자산증가율            0
매출액증가율             0
자기자본증가율            0
총자본증가율             0
정상영업이익증가율          0
장단기금리차             0
cpi                0
업력                 0
누적수익성비율            0
이자부담률              0
부채상환계수             0
현금흐름 대 자산    

In [83]:
# 결측치 제거
df.dropna(axis=0, inplace=True)

In [84]:
print('전체 행:', df.shape[0],'개')
print('전체 기업 수',len(df['회사명'].unique()))
print("결측치 수 : ", df.isna().sum().sum())

전체 행: 10662 개
전체 기업 수 1453
결측치 수 :  0



## 결측치 처리 완료
---

In [85]:
# 무한대 값 확인

print("양의 무한대 :",df[df[["이자부담률"]].isin([np.inf]).any(axis = 1)]["target_3"].value_counts())
print("음의 무한대 :", df[df[["이자부담률"]].isin([-np.inf]).any(axis = 1)]["target_3"].value_counts())

양의 무한대 : target_3
0    548
1      1
Name: count, dtype: int64
음의 무한대 : target_3
0    96
1     3
Name: count, dtype: int64


In [86]:
# 이자부담률 무한대 값 삭제
# 전년도 이자부담률 데이터가 0값이서 데이터 제거(648개)

a = df[df["이자부담률"].isin([np.inf])].index.to_list()
b = df[df["이자부담률"].isin([-np.inf])].index.to_list()
c = a + b
print("이자부담률 무한대 값 행 수 :", len(c))
df.drop(index=c, inplace=True)

이자부담률 무한대 값 행 수 : 648


In [87]:
# 유형자산증가율 무한대 값 삭제
# 전년도 유형자산 데이터가 0값이서 데이터 제거(2개)


a = df[df["유형자산증가율"].isin([np.inf])].index.to_list()
b = df[df["유형자산증가율"].isin([-np.inf])].index.to_list()
c = a + b
print("유형자산증가율 무한대 값 행 수 :", len(c))
df.drop(index=c, inplace=True)

유형자산증가율 무한대 값 행 수 : 2


In [88]:
# 현금흐름 대 매출액 무한대 값 삭제
# 전년도 매출액 데이터가 0값이서 데이터 제거(2개)

a = df[df["현금흐름 대 매출액"].isin([np.inf])].index.to_list()
b = df[df["현금흐름 대 매출액"].isin([-np.inf])].index.to_list()
c = a + b
print("현금흐름 대 매출액 무한대 값 행 수 :", len(c))
df.drop(index=c, inplace=True)

현금흐름 대 매출액 무한대 값 행 수 : 1


In [89]:
# 매출액증가율 무한대 값 삭제
# 전년도 매출액 데이터가 0값이서 데이터 제거

a = df[df["매출액증가율"].isin([np.inf])].index.to_list()
b = df[df["매출액증가율"].isin([-np.inf])].index.to_list()
c = a + b
print("매출액증가율 무한대 값 행 수 :", len(c))
df.drop(index=c, inplace=True)

매출액증가율 무한대 값 행 수 : 3


In [90]:
# ROA변화율 무한대 값 삭제
# 전년도 ROA 데이터가 0값이서 데이터 제거

a = df[df["ROA변화율"].isin([np.inf])].index.to_list()
b = df[df["ROA변화율"].isin([-np.inf])].index.to_list()
c = a + b
print("ROA변화율 무한대 값 행 수 :", len(c))
df.drop(index=c, inplace=True)

ROA변화율 무한대 값 행 수 : 5


In [91]:
# 무한대 값 확인

print("양의 무한대 :",df[df[["이자부담률"]].isin([np.inf]).any(axis = 1)]["target_3"].value_counts())
print("음의 무한대 :", df[df[["이자부담률"]].isin([-np.inf]).any(axis = 1)]["target_3"].value_counts())

양의 무한대 : Series([], Name: count, dtype: int64)
음의 무한대 : Series([], Name: count, dtype: int64)


In [92]:
print("전체 데이터 수 : ", df.shape[0])
print("부실 기업 데이터 수: ",len(df[df["target_3"]==1]))
print("정상 기업 데이터 수 : ",len(df[df["target_3"]==0]))
print("부실 정상 데이터 비율 : ",df["target_3"].value_counts(property))
print("전체 기업 수 : " , len(np.unique(df["회사명"])))
print("부실 기업 수  : ",len(np.unique(df[df["target_1"]==1]["회사명"])))
print("정상 기업 수 : ",len(np.unique(df[df["target_3"]==0]["회사명"])))
print("결측치 수 : ", df.isna().sum().sum())

전체 데이터 수 :  10003
부실 기업 데이터 수:  237
정상 기업 데이터 수 :  9766
부실 정상 데이터 비율 :  target_3
0    0.976307
1    0.023693
Name: proportion, dtype: float64
전체 기업 수 :  1442
부실 기업 수  :  147
정상 기업 수 :  1406
결측치 수 :  0



## 무한대 값 처리 완료
---

In [93]:
df[df["산업군"].str.contains("금융")]

drop_ind = df[df["산업군"].str.contains("금융")].index
df.drop(index=drop_ind, axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)

In [94]:
print("전체 데이터 수 : ", df.shape[0])
print("부실 기업 데이터 수: ",len(df[df["target_3"]==1]))
print("정상 기업 데이터 수 : ",len(df[df["target_3"]==0]))
print("부실 정상 데이터 비율 : ",df["target_3"].value_counts(property))
print("전체 기업 수 : " , len(np.unique(df["회사명"])))
print("부실 기업 수  : ",len(np.unique(df[df["target_1"]==1]["회사명"])))
print("정상 기업 수 : ",len(np.unique(df[df["target_3"]==0]["회사명"])))
print("결측치 수 : ", df.isna().sum().sum())

전체 데이터 수 :  9995
부실 기업 데이터 수:  237
정상 기업 데이터 수 :  9758
부실 정상 데이터 비율 :  target_3
0    0.976288
1    0.023712
Name: proportion, dtype: float64
전체 기업 수 :  1440
부실 기업 수  :  147
정상 기업 수 :  1404
결측치 수 :  0


# 원본 데이터 결측치 처리 완료
---

In [95]:
# 부실 기업 타겟 0인 데이터 삭제  

target_ls = df[df["target_3"]==1]["회사명"].unique().tolist()
drop_target_ls = df[(df["회사명"].isin(target_ls)) & (df["target_3"]==0)].index

df.drop(index=drop_target_ls, axis=0, inplace=True)

print("부실 기업 타겟 0인 데이터 삭제  수 : " ,len(drop_target_ls))

부실 기업 타겟 0인 데이터 삭제  수 :  531


In [96]:
# # 타겟 1인 기업들 0값 지워주기

# # 부실정의 컬럼 
# df['sum'] = df['target_1']+df['target_2']+df['target_3']

# ## 임시 부실데이터저장
# temp = pd.DataFrame()
# for i in df['회사명'].unique():
#     a = df[(df['회사명']==i) & df['sum']!=0]
#     temp = pd.concat([temp,a],axis=0)
# temp.drop(columns=['sum'],axis=1,inplace=True)
# print(temp.shape)

# for i in temp['회사명'].unique():
#     df = df[~(df['회사명']==i)]

# df = pd.concat([df,temp],axis=0)

# df.drop(columns=["sum"], axis=1, inplace=True)
# df.reset_index(drop=True,inplace=True)
# print(df.shape)


In [97]:
print("전체 데이터 수 : ", df.shape[0])
print("부실 기업 데이터 수: ",len(df[df["target_3"]==1]))
print("정상 기업 데이터 수 : ",len(df[df["target_3"]==0]))
print("부실 정상 데이터 비율 : ",df["target_3"].value_counts(property))
print("전체 기업 수 : " , len(np.unique(df["회사명"])))
print("부실 기업 수  : ",len(np.unique(df[df["target_1"]==1]["회사명"])))
print("정상 기업 수 : ",len(np.unique(df[df["target_3"]==0]["회사명"])))
print("결측치 수 : ", df.isna().sum().sum())

전체 데이터 수 :  9464
부실 기업 데이터 수:  237
정상 기업 데이터 수 :  9227
부실 정상 데이터 비율 :  target_3
0    0.974958
1    0.025042
Name: proportion, dtype: float64
전체 기업 수 :  1440
부실 기업 수  :  147
정상 기업 수 :  1293
결측치 수 :  0


In [98]:
df.to_csv("./data/결측치처리완료/코스닥결측치처리완료.csv", encoding="utf-8-sig")